## Import libraries

In [ ]:
import os
import json

import numpy as np
from scipy.spatial.distance import euclidean, hamming
from scipy.stats import spearmanr, pearsonr

In [ ]:
def read_memmap(filepath):
    with open(filepath.replace(".dat", ".conf"), "r") as fin_config:
        memmap_configs = json.load(fin_config)
        return np.memmap(filepath, mode="r", shape=tuple(memmap_configs["shape"]), dtype=memmap_configs["dtype"])

In [ ]:
def format_distance_3bits(a, b):
    a = list(f"{a:03b}")
    b = list(f"{b:03b}")
    return 3*hamming(a, b)

In [ ]:
result_dir = "../../../results"

## Correlation analysis

### Correlation with consistency (best layer)

In [ ]:
os.makedirs(f"correlation_analysis_table", exist_ok=True)

dataset_names = ["CommonsenseQA", "QASC", "100TFQA", "GSM8K", "MMLU-Pro-Law-100Q"]
model_names = ["Phi-3.5-mini-instruct", "Llama-3.1-8B-Instruct"]
prompting_strategies = ["zero-shot", "zero-shot-cot", "few-shot", "few-shot-cot"]

NUM_LAYERS = 32
corr_matrix, p_matrix = np.zeros((len(model_names), len(dataset_names), len(prompting_strategies), NUM_LAYERS)), np.zeros((len(model_names), len(dataset_names), len(prompting_strategies), NUM_LAYERS))
for i, model_name in enumerate(model_names):
    for j, dataset_name in enumerate(dataset_names):
        for k, prompting_strategy in enumerate(prompting_strategies):
            output_dir = f"{result_dir}/{dataset_name}/{model_name}"

            layer_wise_path = os.path.join(output_dir, f"{prompting_strategy}_layer_wise_hidden_states.dat")
            head_wise_path = os.path.join(output_dir, f"{prompting_strategy}_head_wise_hidden_states.dat")

            try:
                layer_wise_hidden_states = read_memmap(layer_wise_path)
            except Exception as e:
                print(e)
                # corr_matrix[i][j][k] = -2
                # p_matrix[i][j][k] = -2
                continue
            num_samples, num_formats, num_layers, hidden_size = layer_wise_hidden_states.shape

            # Prepare pairwise consistency matrix (macro)
            score_path = os.path.join(output_dir, f"{prompting_strategy}_score.json")
            with open(score_path, "r") as fin:
                score = json.load(fin)
            pairwise_consistency_matrix = np.ones((num_formats, num_formats))
            for key, value in score["consistency"].items():
                if key == "mean":
                    continue
                split_key = key.split("_")
                ii, jj = int(split_key[0]), int(split_key[-1])
                pairwise_consistency_matrix[ii][jj] = value
                pairwise_consistency_matrix[jj][ii] = value

            for layer_idx in range(num_layers):
                # Step 1: Prepare input
                X = layer_wise_hidden_states[:,:,layer_idx,:]
                Y = np.tile(np.arange(num_formats), num_samples)
                X_mean = X.mean(axis=0)
                
                # Step 2: Macro analysis
                pairwise_distance_list, pairwise_consistency_list = [], []
                for ii in range(num_formats):
                    for jj in range(num_formats):
                        if ii < jj:
                            dist = euclidean(X_mean[ii], X_mean[jj])
                            pairwise_distance_list.append(dist)
                            pairwise_consistency_list.append(pairwise_consistency_matrix[ii][jj])

                pearson_corr, pearson_p = pearsonr(pairwise_distance_list, pairwise_consistency_list)
                spearman_corr, spearman_p = spearmanr(pairwise_distance_list, pairwise_consistency_list)

                # Print correlation results
                print(f"{dataset_name} / {model_name} / {prompting_strategy}")
                print(f"Layer {layer_idx}")
                print("Macro analysis")
                print(f"Pearson correlation: {pearson_corr:.2f} (p-value: {pearson_p:.3e})")
                print(f"Spearman correlation: {spearman_corr:.2f} (p-value: {spearman_p:.3e})")
                print()

                # if spearman_corr < corr_matrix[i][j][k]:
                corr_matrix[i][j][k][layer_idx] = spearman_corr
                p_matrix[i][j][k][layer_idx] = spearman_p

# prompting_strategies = ["Zero-shot", "Zero-shot CoT", "Few-shot", "Few-shot CoT"]
# # Create LaTeX table string
# latex_code = "\\begin{table}[ht]\n"
# latex_code += "\\centering\n"
# latex_code += "\\caption{Correlation analysis between best layer embedding distance and pairwise consistency. A layer in which the correlation is maximized is selected as the best layer. Each cell represents Spearman correlation coefficient and p-value (in parenthesis).}\n"
# latex_code += "\\label{tab:best_layer_embedding_distance_correlation}\n"
# latex_code += "\\begin{tabular}{c|c|cccc}\n"
# latex_code += "\\toprule\n"
# latex_code += "Model & Task & " + " & ".join(prompting_strategies) + " \\\\\n"
# latex_code += "\\midrule\\midrule\n"

# # Fill the table with data
# for model_idx, model in enumerate(model_names):
#     latex_code += f"\\multirow{{4}}{{*}}{{{model}}}\n"
#     for task_idx, task in enumerate(dataset_names):
#         latex_code += "      "
#         latex_code += f"& {task} "
#         for strat_idx in range(len(prompting_strategies)):
#             corr = corr_matrix[model_idx, task_idx, strat_idx]
#             p = p_matrix[model_idx, task_idx, strat_idx]

#             # Skip conditions
#             if corr < -1.5:
#                 latex_code += f"& - "
#                 continue

#             corr_str = f"{corr:.2f}"
#             p_str = f"{p:.3e}" if p < 0.001 else f"{p:.3f}"
            
#             latex_code += f"& {corr_str} ({p_str}) "
#         latex_code += "\\\\\n"
#     latex_code += "\\midrule\n"

# # Close LaTeX table
# latex_code = latex_code.rstrip("\\midrule\n")  # Remove last midrule
# latex_code += "\\\\\n"
# latex_code += "\\bottomrule\n"
# latex_code += "\\end{tabular}\n"
# latex_code += "\\end{table}"

# # Display the generated LaTeX table code
# print(latex_code)

# with open(f"correlation_analysis_table/best_layer_embedding_distance_correlation_with_consistency_table.txt", "w") as fout:
#     fout.write(latex_code)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

os.makedirs('./embedding_dist_corr_layerwise', exist_ok=True)

# 1. 시각화 설정 및 파라미터 정의
# 폰트 설정 (깨짐 방지)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

# 정의된 파라미터들
# model_names = ['llama-3.1-8B', 'phi-3.5-mini-instruct']
# dataset_names = ['CommonsenseQA', 'GSM8K']
# prompting_strategies = ['zero-shot', 'zero-shot-cot', 'few-shot', 'few-shot-cot']
num_layers = NUM_LAYERS
layers = np.arange(num_layers)

# ⭐️ 핵심 변경 사항: 프롬프팅 전략별로 색상 고정
color_map = {
    'zero-shot': 'tab:blue',
    'zero-shot-cot': 'tab:orange',
    'few-shot': 'tab:green',
    'few-shot-cot': 'tab:red'
}

# 2. ## 실제 데이터로 교체할 부분 ##
# 시연을 위한 임의의 샘플 데이터 생성
# 실제 데이터를 이와 같은 딕셔너리 구조로 준비해주세요.
correlation_data = corr_matrix

print("샘플 데이터 구조 (NumPy Array):")
print(f"전체 데이터 shape: {correlation_data.shape}")
print("-" * 30)


# 3. 시각화 및 파일 저장
for model_idx, model in enumerate(model_names):
    for dataset_idx, dataset in enumerate(dataset_names):
        plt.figure(figsize=(12, 7))
        has_data = False
        
        for strategy_idx, strategy in enumerate(prompting_strategies):
            correlations = correlation_data[model_idx, dataset_idx, strategy_idx, :]
            
            if not np.any(correlations):
                continue
            
            has_data = True
            
            # ⭐️ 핵심 변경 사항: color_map에서 지정된 색상을 사용
            plt.plot(layers, correlations, marker='o', linestyle='-', label=strategy, color=color_map[strategy])
            
        plt.title(f'Model: {model} | Dataset: {dataset}\nLayer-wise Spearman Correlation', fontsize=16)
        plt.xlabel('Layer Number', fontsize=12)
        plt.ylabel('Spearman Correlation Coefficient', fontsize=12)
        plt.ylim(-1.0, 1.0)
        plt.grid(True, linestyle='--', alpha=0.6)
        
        if has_data:
            plt.legend(title='Prompting Strategy', fontsize=10)
            filename = f'embedding_dist_corr_layerwise/{model}_{dataset}_embedding_dist_corr_layerwise.png'
            plt.savefig(filename)
            print(f"'{filename}' 파일이 저장되었습니다.")
            # plt.show()
        else:
            print(f"WARNING: '{model} | {dataset}' 조합에는 유효한 데이터가 없어 그래프를 저장하지 않습니다.")

        plt.close()

In [ ]:
agg_corr_matrix = np.zeros((len(model_names), len(dataset_names), len(prompting_strategies), 6))

for i in range(len(model_names)):
    for j in range(len(dataset_names)):
        for k in range(len(prompting_strategies)):
            agg_corr_matrix[i][j][k][0] = corr_matrix[i][j][k].mean()
            agg_corr_matrix[i][j][k][1] = corr_matrix[i][j][k].std()
            agg_corr_matrix[i][j][k][2] = corr_matrix[i][j][k].min()
            agg_corr_matrix[i][j][k][3] = corr_matrix[i][j][k].max()
            agg_corr_matrix[i][j][k][4] = corr_matrix[i][j][k].argmin()
            agg_corr_matrix[i][j][k][5] = corr_matrix[i][j][k].argmax()

In [ ]:
import numpy as np

# --- 2. LaTeX 테이블 생성 함수 ---

def generate_latex_table(agg_matrix, models, datasets, strategies):
    """
    집계된 데이터 행렬을 바탕으로 LaTeX 테이블 코드를 생성합니다.
    """
    # LaTeX 코드 시작 부분
    latex_code = []
    latex_code.append(r"\begin{table*}[t]")
    latex_code.append(r"    \centering")
    latex_code.append(r"    \caption{Aggregated Spearman Correlation Statistics by Model, Dataset, and Strategy}")
    latex_code.append(r"    \label{tab:agg_stats}")
    latex_code.append(r"    \begin{tabular}{lllrrrrr}")
    latex_code.append(r"        \toprule")
    latex_code.append(r"        \textbf{Model} & \textbf{Dataset} & \textbf{Strategy} & \textbf{Mean Corr.} & \textbf{Std} & \textbf{Min Corr.} & \textbf{Max Corr.} & \textbf{Best Layer} \\")
    latex_code.append(r"        \midrule")

    # 데이터 행 추가
    num_strategies = len(strategies)
    for i, model in enumerate(models):
        # 모델별로 구분선 추가 (첫 모델 제외)
        if i > 0:
            latex_code.append(r"        \midrule")
        
        num_rows_for_model = len(datasets) * num_strategies
        model_str = f"\\multirow{{{num_rows_for_model}}}{{*}}{{{model.replace('_', '-')}}}"

        for j, dataset in enumerate(datasets):
            if j > 0:
                latex_code.append(r"        \cmidrule(l){2-8}")
            
            dataset_str = f"\\multirow{{{num_strategies}}}{{*}}{{{dataset}}}"
            
            for k, strategy in enumerate(strategies):
                stats = agg_matrix[i, j, k]
                mean, std, min_c, max_c, best_layer_idx = stats[0], stats[1], stats[2], stats[3], int(stats[4])

                row = []
                if j == 0 and k == 0: # 모델의 첫 번째 행
                    row.append(f"        {model_str}")
                else:
                    row.append("        ") # 빈 셀

                if k == 0: # 데이터셋의 첫 번째 행
                    row.append(f"{dataset_str}")
                else:
                    row.append("        ") # 빈 셀
                
                row.append(f"{strategy}")
                row.append(f"{mean:.2f}")
                row.append(f"{std:.2f}")
                row.append(f"{min_c:.2f}")
                row.append(f"{max_c:.2f}")
                row.append(f"{best_layer_idx}")
                
                latex_code.append(" & ".join(row) + r" \\")

    # LaTeX 코드 끝 부분
    latex_code.append(r"        \bottomrule")
    latex_code.append(r"    \end{tabular}")
    latex_code.append(r"\end{table*}")
    
    return "\n".join(latex_code)

# --- 3. 함수 실행 및 결과 출력 ---
latex_output = generate_latex_table(agg_corr_matrix, model_names, dataset_names, prompting_strategies)
print(latex_output)